In [ ]:
import urllib.request
from datetime import datetime
import dateutil
import re, os, shutil, logging, json, functools

import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import numpy as np
from scipy.optimize import curve_fit

from ts_viz import TimeSeriesViz

In [ ]:
!mkdir -p figures
!mkdir -p data

In [ ]:
def rolling_mean(df, window=7, extend=True, center=True):
    return df.rolling(window=window, center=center, min_periods=(window // 2) if extend else window).mean()

In [ ]:
import functools

def lazyprop(fn):
    attr_name = '_lazy_' + fn.__name__

    @property
    @functools.wraps(fn)
    def _lazyprop(self):
        if not hasattr(self, attr_name):
            setattr(self, attr_name, fn(self))
        return getattr(self, attr_name)

    return _lazyprop

class DataSet:

    def __init__(self, path, repo='pcm-dpc/COVID-19', date_cols=['data'], index_cols=['data'], resample=False):
        self.repo = repo
        self.path = path
        self.commit_url = f'https://api.github.com/repos/{self.repo}/commits?path={self.path}&page=1&per_page=1'
        self.data_url = f'https://raw.githubusercontent.com/{self.repo}/master/{self.path}'
        self.resample = resample
        self.date_cols = date_cols
        self.index_cols = index_cols
    
    @lazyprop
    def last_modified(self):
        with urllib.request.urlopen(self.commit_url) as url:
            data = json.loads(url.read().decode())
            date = data[0]['commit']['committer']['date']
            utc_date = dateutil.parser.parse(date)
            return utc_date.astimezone(dateutil.tz.gettz('Italy/Rome'))

    @lazyprop
    def df(self):
        df = pd.read_csv(self.data_url, parse_dates=self.date_cols, index_col=self.index_cols)
        if self.resample:
            df = df.resample('D').last()
        return df
        
    def __repr__(self):
        return (f'DataSet\n  repo: {self.repo}\n  path: {self.path}\n  commit_url: {self.commit_url}\n'
                f'  last_modified: {self.last_modified}\n  data_url: {self.data_url}\n  df: {len(self.df)} items')



In [ ]:
def show_cases(cases_df, pop, title='Casi', figsize=(16, 10)):
    columns = ['deceduti', 'terapia_intensiva', 'ricoverati_con_sintomi', 
               'isolamento_domiciliare', 'dimessi_guariti']
    colors = ['tab:purple', 'tab:red', 'tab:orange', 'tab:blue', 'tab:green']
    data = cases_df[columns] / pop
    ax = data.plot(kind='area', title=title, color=colors, figsize=figsize)
#     fig, ax = plt.subplots(figsize=figsize)
#     ax.stackplot(cases_df.index, cases_df[columns].T, colors=colors)
    ax.set_title(title)
    ax.xaxis.grid(True, which='major')
    ax.yaxis.grid(True, which='major')
    locator = mdates.MonthLocator(interval=1)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
    ax.xaxis.set_label_text('')
    ax.axhline(0.0, linewidth=2, color='white')
    ax.legend()
    return ax

In [ ]:
class Regional:
    
    def __init__(self, figures_path=None):
        self.figures_path = figures_path
        self.dataset = DataSet('dati-regioni/dpc-covid19-ita-regioni.csv')
        self.reg_pop = pd.read_csv('data/popolazione_regioni.csv', names=['region', 'population'], 
                                   index_col='region')
        if self.figures_path:
            self.cases_path = os.path.join(self.figures_path, 'cases')
            os.makedirs(self.cases_path, exist_ok=True)
        else:
            self.cases_path = None
    
    @staticmethod
    def slug(region):
        return re.sub("\W", '', region).lower()
    
    def cases_figure_filename(self, region_name, last_modified=True):
        if not self.cases_path:
            return None
        filename_elems = [Regional.slug(region_name)]
        if last_modified:
            filename_elems.append(f'{self.dataset.last_modified:%Y%m%d}')
        filename = '-'.join(filename_elems) + '.png'
        return os.path.join(self.cases_path, filename)
    
    def total_cases_figure_filename(self, last_modified=True):
        if not self.cases_path:
            return None
        filename_elems = ['00-total']
        if last_modified:
            filename_elems.append(f'{self.dataset.last_modified:%Y%m%d}')
        filename = '-'.join(filename_elems) + '.png'
        return os.path.join(self.cases_path, filename)
    
    def save_figure(self, filenames, show=True, close=True):
        if filenames:
            plt.savefig(filenames[0])
            for filename in filenames[1:]:
                shutil.copyfile(filenames[0], filename)
        if show:
            plt.show()
        if close:
            plt.close()

    def cases_figure(self, region_name, show=False, ylim=(-1200, 500), figsize=(16, 10)):
        cases_df = self.dataset.df[self.dataset.df['denominazione_regione'] == region_name]
        cases_df = cases_df.resample('D').last()
        cases_df['deceduti'] = -cases_df['deceduti']
        cases_df['dimessi_guariti'] = -cases_df['dimessi_guariti']
        pop = self.reg_pop.at[region_name, 'population'] / 100000
        ax = show_cases(cases_df, pop, 
                        title=f'Casi per 100.000 persone nella regione di {region_name}\n'
                              f'(dati aggiornati: {self.dataset.last_modified:%d/%m/%Y %H:%M:%S})', 
                        figsize=figsize)
        ax.set_ylim(ylim)
        filenames = []
        fn_date = self.cases_figure_filename(region_name)
        if fn_date:
            filenames.append(fn_date)
        fn_last = self.cases_figure_filename(region_name, last_modified=False)
        if fn_last:
            filenames.append(fn_last)
        ax.xaxis.grid(True, which='major')
        ax.yaxis.grid(True, which='major')
        locator = mdates.MonthLocator(interval=1)
        ax.xaxis.set_major_locator(locator)
        ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
        ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
        ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
        ax.legend(loc='upper left')
        self.save_figure(filenames, show)
    
    def region_names(self):
        return self.dataset.df['denominazione_regione'].unique()
    
    def all_regions_cases_figure(self, show_regions=[], ylim=(-1200, 500), figsize=(16, 10)):
        for region_name in show_regions:
            self.cases_figure(region_name, show=(region_name in show_regions), ylim=ylim, figsize=figsize)

    def regional_totals(self, per_population=False, column='totale_casi', diff=True, window=7):
        regional_totals = self.dataset.df.pivot(columns='denominazione_regione', values=column)
        regional_totals['Trentino-Alto Adige'] = \
            regional_totals['P.A. Bolzano'] + regional_totals['P.A. Trento']
        regional_totals = regional_totals.drop(['P.A. Bolzano', 'P.A. Trento'], axis='columns')
        regional_totals = regional_totals[sorted(regional_totals.columns)]
        regional_totals = regional_totals.resample('D').last()
        if per_population:
            regional_totals = regional_totals.div(self.reg_pop['population']) * 100000
        if diff:
            regional_totals = (regional_totals - regional_totals.shift()).clip(0, None)
        regional_totals = rolling_mean(regional_totals, window, extend=False).dropna()
        regional_totals = regional_totals.sort_values(axis='columns', by=regional_totals.index[-1], 
                                                      ascending=False)
        return regional_totals
            
    def total_cases_figure(self, show=True, per_population=False, column='totale_casi', diff=True, window=7, 
                           ylim=(0, None), xlim=(None, None), figsize=(16, 10)):
        regional_totals = self.regional_totals(per_population, column, diff, window)
        plot_title = f'{column} {"giornalieri" if diff else "casi"} ' \
                     f'{"per 100.000 persone " if per_population else ""}' \
                     f'in Italia per regione\n' \
                     f'(dati aggiornati: {self.dataset.last_modified:%d/%m/%Y %H:%M:%S})'
        fig, ax = plt.subplots(figsize=figsize)
        linestyles = ['-', '--']
        if per_population:
            for idx, region in enumerate(regional_totals.columns):
                ax.plot(regional_totals.index, regional_totals[region], label=region, ls=linestyles[idx//10])
            ax.set_title(plot_title)
        else:
            ax = regional_totals.plot(kind='line' if per_population else 'area', figsize=figsize, 
                                      title=plot_title)

        ax.xaxis.grid(True, which='major')
        ax.yaxis.grid(True, which='major')
        locator = mdates.WeekdayLocator(byweekday=mdates.MO)
        ax.xaxis.set_major_locator(locator)
        ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
        ax.xaxis.set_label_text('')
        ax.xaxis.set_minor_locator(mdates.DayLocator())
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        ax.legend(title=None, ncol=2)
        filenames = []
        fn_date = self.total_cases_figure_filename()
        if fn_date:
            filenames.append(fn_date)
        fn_last = self.total_cases_figure_filename(last_modified=False)
        if fn_last:
            filenames.append(fn_last)
        self.save_figure(filenames, show)

In [ ]:
# func = lambda x, a, b: a * np.exp(b * x) 
# func_name = 'exp'

func = lambda t, K, x, t0, b: K * np.power(t, x) * np.exp(-t / t0) + b
func_name = 'power'

def show_fit(series, func, func_name, title=None, pred=7, figsize=(10, 6), log=False, ax=None, window=None,
             fit_color = 'tab:red', ma_color = 'tab:orange'):
    x = np.arange(len(series))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        popt, pcov = curve_fit(func, x, series.values, p0=(1, 1, 5, 0), maxfev=5000)
    index = pd.date_range(series.index.min(), periods=(len(series) + pred), freq='D')
    y = func(np.arange(len(index)), *popt)
    fit = pd.Series(y, index=index)
    
    err = np.sqrt(np.sum((fit[:-pred] - series) ** 2) / len(series))
    print(f'Fit error: {err:.02f}')
    
    print(f'Predicted volume error: {1 - fit[:-pred].sum() / series.sum():.02%}')
    
    lw = 3
    
    if not ax:
        fig, ax = plt.subplots(figsize=figsize)
    ax.plot(index, y, label=f'{func_name} fit', color=fit_color, lw=lw)
    ax.bar(series.index, series, label=series.name)
    argmax = index[np.argmax(fit)]
    ax.axvline(argmax, color=fit_color, lw=lw)
    if window:
        series_ma = series.rolling(window=window, center=True).mean()
        ax.plot(series_ma.index, series_ma, label=f'media mobile ({window} giorni)', color=ma_color, lw=lw)
    
    locator = mdates.MonthLocator(interval=1)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
    ax.yaxis.grid(True, which='major')
    if log:
        ax.set_yscale('log')
        ax.set_ylim((0.1, series.max() * 1.1))
    else:
        ax.set_ylim((0, None))
    if title:
        ax.set_title(title)
    plt.legend(loc='upper left')
    return popt, pcov, fit, argmax

# Casi regionali

In [ ]:
regional_ds = DataSet('dati-regioni/dpc-covid19-ita-regioni.csv')
print(regional_ds)
regional_ds.df[regional_ds.df['denominazione_regione'] == 'Lombardia'].tail(n=10)
print(regional_ds.df['denominazione_regione'].unique())

In [ ]:
national = DataSet('dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv', resample=True)
print(national)
national.df.tail(n=10)

In [ ]:
national_df = national.df.resample('D').last()
national_df['deceduti'] = -national_df['deceduti']
national_df['dimessi_guariti'] = -national_df['dimessi_guariti']
show_cases(national_df, 1, title=f'Casi in Italia\n'
           f'(dati aggiornati: {national.last_modified:%d/%m/%Y %H:%M:%S})', figsize=(16, 10))
plt.show()

In [ ]:
days = 35
fig, ax = plt.subplots(figsize=(16, 10))
pos = national_df['totale_positivi']
ax.plot(pos.index, pos, label=f'totale positivi', color='tab:blue', lw=2)

def plot_rol(days, color, ls='-'):
    rol = national_df['nuovi_positivi'].rolling(window=days).sum()
    ax.plot(rol.index, rol, label=f'somma mobile ({days}g) positivi giornalieri', color=color, lw=2, ls=ls)

plot_rol(35, 'tab:orange')
plot_rol(14, 'tab:orange', '--')
ax.legend()
ax.set_title('Stima della durata della malattia a base di totale positivi / nuovi positivi giornalieri')
ax.set_xlim(('2020-08-01', None))
plt.show()

In [ ]:
def showdiff(df, n=10):
    return (df - df.shift()).tail(n=n)

In [ ]:
national_df['totale_positivi'].tail(n=10)

In [ ]:
showdiff(national_df['totale_positivi'])

In [ ]:
national_df['totale_casi'].tail(n=10)

In [ ]:
showdiff(national_df['totale_casi'])

In [ ]:
-national_df['deceduti'].tail()

In [ ]:
regional = Regional()
regional.dataset.df[regional_ds.df['denominazione_regione'] == 'Lombardia'].tail()

In [ ]:
last_date = regional.dataset.df.index.max()
regional.total_cases_figure(per_population=True, column='totale_positivi', 
                            diff=False, xlim=('2020-10-01', last_date.strftime('%Y-%m-%d')), window=1)
plt.show()

In [ ]:
window = 7
case_last_date = (last_date - pd.Timedelta(days=window // 2)).strftime('%Y-%m-%d')
regional.total_cases_figure(column='totale_casi', xlim=('2020-10-01', case_last_date), ylim=(0, None),
                            diff=True, per_population=True, window=window)
plt.show()

In [ ]:
regional.all_regions_cases_figure(show_regions=[
    'Lombardia', 'Piemonte', 'Veneto', 'Sardegna', 'Lazio', 'Liguria', 'Umbria', 
    'Toscana', 'Emilia-Romagna', 'Puglia', 'Campania', 'Valle d\'Aosta'], ylim=(-3000, 2000))

In [ ]:
national_tot_df = national.df.copy()
national_tot_df['deceduti'] = -national_tot_df['deceduti']
national_tot_df['dimessi_guariti'] = -national_tot_df['dimessi_guariti']
show_cases(national_tot_df, 1, title=f'Casi in Italia\n(dati aggiornati: {national.last_modified:%d/%m/%Y %H:%M:%S})')
plt.show()

# Lombardia

## Casi totali in Lombardia

In [ ]:
mkdir -p figures/overview

In [ ]:
from importlib import reload
import ts_viz
reload(ts_viz)
from ts_viz import TimeSeriesViz, OverviewViz

In [ ]:
def get_series(region, name):
    region_df = regional_ds.df[regional_ds.df['denominazione_regione'] == region]
    return region_df[name].resample('D').last()

In [ ]:
lombardia_df = regional_ds.df[regional_ds.df['denominazione_regione'] == 'Lombardia']
lombardia_overview_viz = OverviewViz('Lombardia', lombardia_df, regional_ds.last_modified, fig_folder='figures/overview')
lombardia_overview_viz.show_overview()

In [ ]:
showdiff(lombardia_df['deceduti'], n=10).astype(int)

In [ ]:
showdiff(lombardia_df['totale_casi'], n=20).astype(int)

In [ ]:
lombardia_df['totale_positivi'].plot()

In [ ]:
lombardia_df['terapia_intensiva'].tail(n=10)

In [ ]:
showdiff(lombardia_df['terapia_intensiva'])

In [ ]:
lomb_casi_diff = lombardia_df['casi_testati'] - lombardia_df['casi_testati'].shift()
lomb_tamp_diff = lombardia_df['tamponi'] - lombardia_df['tamponi'].shift()
(lomb_tamp_diff / lomb_casi_diff).tail(n=10)

In [ ]:
italia_df = national.df
italia_overview_viz = OverviewViz('Italia', national.df, national.last_modified, fig_folder='figures/overview')
italia_overview_viz.show_overview()

In [ ]:
showdiff(italia_df['totale_casi'])

In [ ]:
showdiff(italia_df['deceduti'])

In [ ]:
italia_df['terapia_intensiva'].tail(n=10)

In [ ]:
lombardia_active = get_series('Lombardia', 'totale_positivi')
lombardia_active[-1] / lombardia_active[-14]

In [ ]:
lombardia_totale = get_series('Lombardia', 'totale_casi')
lombardia_totale.name = 'totali Lombardia'
lombardia_totale_viz = TimeSeriesViz(series=lombardia_totale, last_modified=regional_ds.last_modified)
lombardia_totale_viz.logger.setLevel(logging.INFO)
lombardia_tamp = get_series('Lombardia', 'casi_testati')
# lombardia_tamp = get_series('Lombardia', 'tamponi')

In [ ]:
lombardia_totale.tail()

In [ ]:
lombardia_tamp.tail().astype(int)

In [ ]:
showdiff(lombardia_tamp)

In [ ]:
lomb_tot_new = lombardia_totale - lombardia_totale.shift()
lomb_tamp_new = lombardia_tamp - lombardia_tamp.shift()
lomb_tamp_rate = lomb_tot_new / lomb_tamp_new

In [ ]:
def tamp_rate(region, window=1, extend=False, ylim=(0, 100)):
    tamp = get_series(region, 'casi_testati')
#     tamp = get_series(region, 'tamponi')
    tot = get_series(region, 'totale_casi')

    tot_new = rolling_mean(tot - tot.shift(), window, extend=extend)
    tamp_new = rolling_mean(tamp - tamp.shift(), window, extend=extend)
    tamp_rate = tot_new / tamp_new * 100
    
    fig, ax = plt.subplots(figsize=(16, 10))
    tamp_rate.plot()
    ax.set_ylim(ylim)
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(decimals=0))
    
    ax.yaxis.grid(True, which='major')
    ax.set_title(f'Percentuale positivi dei casi testati in {region}')
    ax.set_xlim(('2020-09-01', None))
    plt.show()
    print(tamp_rate.tail(n=10))

In [ ]:
tamp_rate('Lombardia', ylim=(0, 80))

In [ ]:
tamp_rate('Veneto', window=1, ylim=(0, 100))

In [ ]:
# tamp_rate('Sardegna')

In [ ]:
# tamp_rate('Lazio')

In [ ]:
# tamp_rate('Campania')

In [ ]:
tamp_rate('Liguria')

In [ ]:
# tamp_rate('Umbria', ylim=(0, 100))

In [ ]:
# tamp_rate('Toscana')

In [ ]:
tamp_rate('Emilia-Romagna')

In [ ]:
tamp_rate('Valle d\'Aosta', ylim=(0, 100))

In [ ]:
# tamp_rate('Calabria', ylim=(0, 80))

In [ ]:
# lombardia_totale_viz.show_series(title='Casi totali in Lombardia', save_fig=True, save_csv=True)

In [ ]:
lombardia_totale_viz.show_new(title='Nuovi casi giornalieri in Lombardia')

In [ ]:
(lombardia_totale_viz.series - lombardia_totale_viz.series.shift()).tail()

In [ ]:
from matplotlib.dates import date2num, datestr2num
fig, ax = plt.subplots(figsize=(16, 10))
lombardia_totale = get_series('Lombardia', 'totale_casi')
# lombardia_tamp = get_series('Lombardia', 'tamponi')
lombardia_tamp = get_series('Lombardia', 'casi_testati')

diff_tot = (lombardia_totale - lombardia_totale.shift())
diff_tamp = (lombardia_tamp - lombardia_tamp.shift())
ax.bar(date2num(diff_tot.index) - 0.2, diff_tot, width=0.4, label='casi positivi')
ax.bar(date2num(diff_tamp.index) + 0.2, diff_tamp, width=0.4, label='casi testati')
ax.set_title('Nuovi casi giornalieri e numero tamponi giornalieri in Lombardia')
ax.yaxis.grid(True, which='major')
locator = mdates.WeekdayLocator(byweekday=mdates.MO)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
ax.set_ylim(0, None)
ax.legend()
ax.set_xlim((datestr2num('2020-10-01'), date2num(diff_tot.index + pd.Timedelta(days=0.5)).max()))
plt.show()

In [ ]:
showdiff(lombardia_tamp)

In [ ]:
data = {
    'diff_tamp': diff_tamp,
    'diff_tot': diff_tot,
    'tamp_for_last': diff_tamp / diff_tamp[-1],
    'norm_tamp': (diff_tot / (diff_tamp / diff_tamp[-1])),
    'tamp_rate': diff_tamp / diff_tot,
    'pos_rate': (diff_tot / diff_tamp).apply('{:.0%}'.format)
}
pd.DataFrame(data).tail(n=10)

In [ ]:
title = 'Nuovi casi giornalieri in Lombardia, normalizzata al numero dei tamponi'
sma = 7
fig, ax = TimeSeriesViz.config_axis(figsize=(16, 10), xgrid=False, title=title)
norm_tamp = diff_tot / (diff_tamp / diff_tamp[-1])
ax.bar(norm_tamp.index, norm_tamp, label='casi')

norm_tamp_sma = norm_tamp.rolling(sma, center=True).mean()
ax.plot(norm_tamp_sma.index, norm_tamp_sma, color='tab:red', lw=2, label=f'media mobile ({sma} giorni)')

valid_index = norm_tamp[~norm_tamp.isna()].index
ax.set_xlim((valid_index.min() + pd.Timedelta(days=.5), valid_index.max() + pd.Timedelta(days=.5)))
# ax.get_yaxis().set_visible(False)
ax.set_ylim((0, None))
ax.set_xlim(('2020-09-01', None))
locator = mdates.MonthLocator(interval=1)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
ax.legend(loc='upper left')
plt.show()

In [ ]:
diff_tot.tail()

In [ ]:
# fig, ax = plt.subplots(figsize=(16, 10))
# norm_sum = norm_tamp.expanding().sum()
# ax = norm_sum.plot()
# ax.set_xlim(('2020-04-15', None))
# plt.show()

In [ ]:
# lombardia_totale_viz.show_growth_factor(title='Tasso di crescita per i casi totali in Lombardia', 
#                                         raw=True, sma=False, ema=True, save_fig=True, save_csv=True, window=7, ylim=(0, 3))

## Casi decessi in Lombardia

In [ ]:
lombardia_deaths = regional_ds.df[regional_ds.df['denominazione_regione'] == 'Lombardia']['deceduti'].resample('D').last()
lombardia_deaths.name = 'deceduti Lombardia'
lombardia_deaths_viz = TimeSeriesViz(series=lombardia_deaths, last_modified=regional_ds.last_modified)
lombardia_deaths_viz.logger.setLevel(logging.INFO)

In [ ]:
lombardia_deaths_viz.show_series(title='Casi deceduti in Lombardia')

In [ ]:
fig, ax = lombardia_deaths_viz.show_new(title='Nuovi casi deceduti in Lombardia')
ax.set_xlim(('2020-09-01', None))
ax.set_ylim((0, 225))
plt.show()

In [ ]:
# lomb_death = lombardia_deaths_viz.series - lombardia_deaths_viz.series.shift()
# lomb_death[lomb_death >= lomb_death[-1]].tail(n=10)

In [ ]:
showdiff(lombardia_deaths_viz.series, n=10).astype(int)

In [ ]:
lombardia_ti = get_series('Lombardia', 'terapia_intensiva')

In [ ]:
window = 7
fig, ax = TimeSeriesViz.config_axis(figsize=(16, 10), xgrid=False, title=title)
norm_tamp = (diff_tot / (diff_tamp / diff_tamp.mean()))['2020-02-01':]
norm_tamp = rolling_mean(norm_tamp, window, extend=False)
color = 'tab:blue'
ax.plot(norm_tamp.index, norm_tamp, color=color, label='nuovi casi giornalieri, ponderata ai tamponi', lw=2)
ax.set_xlim((norm_tamp.index.min() + pd.Timedelta(days=.5), norm_tamp.index.max() + pd.Timedelta(days=.5)))
ax.set_ylabel('nuovi casi in Lombardia (ponderata al numero dei tamponi)', color=color)
ax.tick_params(axis='y', labelcolor=color)
ax.set_ylim((0, None))
lombardia_deaths_diff = lombardia_deaths - lombardia_deaths.shift()
lombardia_deaths_diff = rolling_mean(lombardia_deaths_diff, window, extend=False)
ltid = lombardia_ti - lombardia_ti.shift()
ltid = rolling_mean(ltid, window, extend=False)
ltids = ltid.shift(-6)
sh = -6
ldds = lombardia_deaths_diff.shift(sh)
ax2 = ax.twinx()
color = 'tab:red'
ax2.plot(ldds.index, ldds, color=color, label='deceduti', lw=2)
ax2.plot(ltids.index, ltids, color='tab:orange', label='terapia intensiva', lw=2)
ax2.set_ylabel(f'nuovi decessi e casi in TI in Lombardia (spostato {sh} giorni)', color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.set_ylim((0, None))
ax.set_title(f'Nuovi casi (ponderata al numero dei tamponi) e decessi (spostato {sh} giorni) in Lombardia')
locator = mdates.DayLocator(interval=7)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
ax.xaxis.grid(True, which='major')
ax.set_xlim(('2020-02-01', None))
ax.set_ylim(0, 5000)
ax2.set_ylim(0, 200)
plt.legend(loc='upper left')
plt.show()

In [ ]:
showdiff(national.df['deceduti'])

In [ ]:
get_series('Lombardia', 'totale_casi').tail()

In [ ]:
get_series('Lombardia', 'deceduti').tail()

In [ ]:
coefs = {}

In [ ]:
import warnings

# region = 'Lombardia'
# lombardia_deaths = get_series(region, 'deceduti')

# observed = (lombardia_deaths - lombardia_deaths.shift())[1:]

# popt, pcov, fit = show_fit(observed, func, func_name, figsize=(16, 10), pred=30, 
#                            title=f'Nuovi deceduti giornalieri in {region} + {func_name} '
#                                  f'fit\n$n(t)=Kt^{{x}}e^{{-t/t_0}}$')
# # popt, pcov, ymax = fit_curve(observed, func, func_name, pred=7)
# K, x, t0 = popt
# coefs['Lombardia deceduti'] = popt
# print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
# print(f'first day: {observed.index[0]:%Y-%m-%d}')
# print(f'peak as of fit: {fit.index[np.argmax(fit)]:%Y-%m-%d}')

In [ ]:
from scipy.optimize import curve_fit
from matplotlib import cm
import numpy as np

func = lambda t, K, x, t0, b: K * np.power(t, x) * np.exp(-t / t0) + b
func_name = 'power'

# func = lambda x, a, b: a * np.exp(b * x) 

def fit_curves(series, func, name, lookback=8, cmap='Reds'):
    colormap = cm.get_cmap(cmap)

    for i in range(0, lookback + 1):
        to_fit = series[:-i] if i > 0 else series
        x = np.arange(len(to_fit))
        popt, pcov = curve_fit(func, x, to_fit.values)
        plt.plot(series.index, func(np.arange(len(series)), *popt), 
                 label=f'{name} fit -{i} days', c=colormap(i/lookback))

# fig, ax = plt.subplots(figsize=(10, 6))

# fit_curves(lombardia_deaths, func, func_name, lookback=12, cmap='winter')
# # plt.plot(lombardia_deaths.index, lombardia_deaths.values)
# ax = lombardia_deaths.plot(c='r', lw=3)
# ax.set_title(f'Decessi in Lombardia + {func_name} fit')
# plt.legend()

In [ ]:
# lomb_diff = (lombardia_totale - lombardia_totale.shift())[1:]
# fig, ax = plt.subplots(figsize=(10, 6))
# fit_curves(lomb_diff, func, func_name, cmap='winter')
# plt.bar(lombardia_totale.index, lombardia_totale.values)
# ax.set_title('Casi totali in Lombardia + exp fit')
# plt.legend()

In [ ]:
def get_normalized_new(region, start_day=None):
    total = get_series(region, 'totale_casi')
    tests = get_series(region, 'casi_testati')
#     tests = get_series(region, 'tamponi')
    diff_total = (total - total.shift())
    diff_tests = (tests - tests.shift())
    normalized_total = (diff_total / (diff_tests / diff_tests[-1]))[1:]
    if start_day:
        normalized_total = normalized_total[start_day:]
    return normalized_total

region = 'Lombardia'
normalized_new = get_normalized_new(region, start_day='2020-09-01')
# normalized_new -= 500
# normalized_new['2020-02-27'] = (normalized_new['2020-02-25'] + normalized_new['2020-02-28']) / 2
# normalized_new['2020-02-26'] = (normalized_new['2020-02-25'] + normalized_new['2020-02-27']) / 2
# normalized_new['2020-03-05'] = (normalized_new['2020-03-04'] + normalized_new['2020-03-06']) / 2
# normalized_new['2020-03-09'] = (normalized_new['2020-03-08'] + normalized_new['2020-03-10']) / 2
normalized_new.name = 'Nuovi casi giornalieri ponderati ai casi testati'
popt, pcov, fit, argmax = show_fit(normalized_new, func, func_name, pred=15, figsize=(16, 10),
                                   title=f'{normalized_new.name} in {region} + {func_name} fit\n'
                                         f'$n(t)=Kt^{{x}}e^{{-t/t_0}}+b$', 
                                   window=7, ma_color='tab:orange')
K, x, t0, b = popt
coefs['Lombardia casi_norm'] = popt
print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}, b = {b:.02f}')
print(f'first day: {normalized_new.index[0]:%Y-%m-%d}')
print(f'peak as of fit: {argmax:%Y-%m-%d}')
popt

In [ ]:
tests = get_series(region, 'casi_testati')
tests_diff = tests - tests.shift()
tamp = get_series(region, 'tamponi')
tamp_diff = tamp - tamp.shift()
(tests_diff/tamp_diff).rolling(window=7).mean()['2020-05-15':].plot(figsize=(16, 10))
plt.title(f'Percentuale dei nuovi casi testati tra i tamponi in {region}')

In [ ]:
region = 'Veneto'
# veneto_deaths = get_series(region, 'deceduti')

# fig, ax = plt.subplots(figsize=(16, 10))
# observed = (veneto_deaths - veneto_deaths.shift())[4:]
# popt, pcov, y = show_fit(observed, func, func_name, title=f'Nuovi deceduti giornalieri in {region} + {func_name} fit', ax=ax)
# ax.set_ylim((0, 50))
# K, x, t0 = popt
# coefs['Veneto deceduti'] = popt
# print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
# print(f'first day: {observed.index[0]:%Y-%m-%d}')
# print(f'peak as of fit: {observed.index[np.argmax(y)]:%Y-%m-%d}')

In [ ]:
region = 'Veneto'
normalized_new = get_normalized_new(region, start_day='2020-09-01')
# normalized_new['2020-03-09'] = (normalized_new['2020-03-08'] + normalized_new['2020-03-10']) / 2
# normalized_new['2020-03-17'] = (normalized_new['2020-03-16'] + normalized_new['2020-03-18']) / 2
fig, ax = plt.subplots(figsize=(16, 10))
normalized_new.name = 'nuovi casi ponderati ai tamponi'
popt, pcov, fit, argmax = show_fit(normalized_new, func, func_name, ax=ax, pred=30,
                                   title=f'Nuovi casi normalizzati giornalieri in {region} + {func_name} '
                                         f'fit\n$n(t)=Kt^{{x}}e^{{-t/t_0}}+b$', window=14)
ax.set_ylim((0, 5000))
K, x, t0, b = popt
coefs['Veneto casi_norm'] = popt
print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}, b = {b:.02f}')
print(f'first day: {normalized_new.index[0]:%Y-%m-%d}')
print(f'peak as of fit: {argmax:%Y-%m-%d}')

In [ ]:
region = 'Piemonte'
# veneto_deaths = get_series(region, 'deceduti')

# observed = (veneto_deaths - veneto_deaths.shift())[4:]
# popt, pcov, y = show_fit(observed, func, func_name, title=f'Nuovi deceduti giornalieri in {region} + {func_name} fit')
# K, x, t0 = popt
# coefs['Piemonte deceduti'] = popt
# print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
# print(f'first day: {observed.index[0]:%Y-%m-%d}')
# # print(f'peak as of fit: {observed.index[np.argmax(y)]:%Y-%m-%d}')

In [ ]:
region = 'Piemonte'
normalized_new = get_normalized_new(region, start_day='2020-09-01')
# normalized_new = normalized_new[3:]
# normalized_new['2020-03-05'] = (normalized_new['2020-03-04'] + normalized_new['2020-03-06']) / 2
# normalized_new['2020-03-13'] = (normalized_new['2020-03-13'] + normalized_new['2020-03-14']) / 2
# normalized_new['2020-03-09'] = 0
fig, ax = plt.subplots(figsize=(16, 10))
normalized_new.name = 'nuovi casi ponderati ai tamponi'

casi = get_series(region, 'totale_casi')
observed = (casi - casi.shift())[1:].clip(0, np.inf)

popt, pcov, fit, argmax = show_fit(normalized_new, func, func_name, ax=ax, pred=30,
                                   title=f'Nuovi casi giornalieri in {region} + {func_name} '
                                         f'fit\n$n(t)=Kt^{{x}}e^{{-t/t_0}}+b$', window=14, ma_color='tab:orange')
K, x, t0, b = popt
coefs['Piemonte casi_norm'] = popt
print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}, b = {b:.02f}')
print(f'first day: {normalized_new.index[0]:%Y-%m-%d}')
print(f'peak as of fit: {argmax:%Y-%m-%d}')

In [ ]:
region = 'Emilia-Romagna'
# deaths = get_series(region, 'deceduti')

# observed = (deaths - deaths.shift())[1:]
# popt, pcov, y = show_fit(observed, func, func_name, title=f'Nuovi deceduti giornalieri in {region} + {func_name} fit')
# K, x, t0 = popt
# coefs['Emilia-Romagna deceduti'] = popt
# print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
# print(f'first day: {observed.index[0]:%Y-%m-%d}')
# print(f'peak as of fit: {observed.index[np.argmax(y)]:%Y-%m-%d}')

In [ ]:
normalized_new = get_normalized_new(region, start_day='2020-10-01')
# normalized_new['2020-03-03'] = (normalized_new['2020-03-02'] + normalized_new['2020-03-04']) / 2
# normalized_new['2020-03-29'] = (normalized_new['2020-03-28'] + normalized_new['2020-03-30']) / 2
# normalized_new['2020-03-30'] = (normalized_new['2020-03-29'] + normalized_new['2020-03-31']) / 2
# normalized_new = normalized_new.clip(0)
normalized_new.name = 'nuovi casi ponderati ai tamponi'
fig, ax = plt.subplots(figsize=(16, 10))
popt, pcov, fit, argmax = show_fit(normalized_new, func, func_name, ax=ax,
                                   title=f'Nuovi casi normalizzati giornalieri in {region} + '
                                         f'{func_name} fit\n$n(t)=Kt^{{x}}e^{{-t/t_0}}+b$', window=7, pred=30)
K, x, t0, b = popt
coefs['Emilia-Romagna casi_norm'] = popt
print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
print(f'first day: {normalized_new.index[0]:%Y-%m-%d}')
print(f'peak as of fit: {argmax:%Y-%m-%d}')

In [ ]:
# casi = get_series(region, 'totale_casi')

# observed = (casi - casi.shift())[1:]
# popt, pcov, y = show_fit(observed, func, func_name, title=f'Nuovi casi giornalieri in {region} + {func_name} fit')
# K, x, t0 = popt
# print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
# print(f'first day: {observed.index[0]:%Y-%m-%d}')
# print(f'peak as of fit: {observed.index[np.argmax(y)]:%Y-%m-%d}')

In [ ]:
# region = 'Puglia'
# deaths = get_series(region, 'deceduti')

# observed = (deaths - deaths.shift())[14:]
# popt, pcov, y = show_fit(observed, func, func_name, title=f'Nuovi deceduti giornalieri in {region} + {func_name} fit')
# K, x, t0 = popt
# coefs['Puglia deceduti'] = popt
# print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
# print(f'first day: {observed.index[0]:%Y-%m-%d}')
# print(f'peak as of fit: {observed.index[np.argmax(y)]:%Y-%m-%d}')

In [ ]:
# casi = get_series(region, 'totale_casi')

# observed = (casi - casi.shift())[1:]
# popt, pcov, y = show_fit(observed, func, func_name, title=f'Nuovi casi giornalieri in {region} + {func_name} fit')
# K, x, t0 = popt
# print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
# print(f'first day: {observed.index[0]:%Y-%m-%d}')
# print(f'peak as of fit: {observed.index[np.argmax(y)]:%Y-%m-%d}')

In [ ]:
# coefs_df = pd.DataFrame.from_records(coefs).T
# coefs_df.columns = ['K', 'x', 't0']
# coefs_df[coefs_df.index.to_series().str.endswith('casi_norm')]

In [ ]:
# lombardia_deaths_viz.show_growth_factor(title='Tasso di crescita dei casi deceduti in Lombardia', 
#                                         sma=False, save_fig=True, save_csv=True, ylim=(0, 2))

## Casi in terapia intensiva in Lombardia

In [ ]:
# lombardia_ti = regional_ds.df[regional_ds.df['denominazione_regione'] == 'Lombardia']['terapia_intensiva'].resample('D').last()
lombardia_ti.name = 'ti Lombardia'
lombardia_ti_viz = TimeSeriesViz(series=lombardia_ti, last_modified=regional_ds.last_modified)
lombardia_ti_viz.logger.setLevel(logging.INFO)

In [ ]:
fig, ax = lombardia_ti_viz.show_series(title='Casi in terapia intensiva in Lombardia')
# ax.set_xlim(('2020-09-01', None))
# ax.set_ylim((0, 2000))

In [ ]:
popt, pcov, fit, argmax = show_fit(lombardia_ti_viz.series['2020-09-15':], func, func_name, pred=60, 
                                   figsize=(16,10))
print('Peak as of fit:', argmax)
td = (argmax - pd.Timestamp('2020-09-15')).days
print('Peak value:', int(func(td, *popt)))
plt.title('Totale casi in terapia intensiva in Lombardia + power fit')
plt.show()

In [ ]:
lombardia_ti_viz.show_new(title='Nuovi casi in terapia intensiva in Lombardia')

In [ ]:
print('Nuovi casi giornalieri in TI in Lombardia:')
showdiff(lombardia_ti_viz.series).astype(int).to_frame()

In [ ]:
print('Totale casi in TI in Lombardia:')
lombardia_ti_viz.series.to_frame().tail(n=10)

In [ ]:
lombardia_osp = regional_ds.df[regional_ds.df['denominazione_regione'] == 'Lombardia']['totale_ospedalizzati'].resample('D').last()
lombardia_osp.name = 'ospedalizzati Lombardia'
lombardia_osp_viz = TimeSeriesViz(series=lombardia_osp['2020--01':], last_modified=regional_ds.last_modified)
lombardia_osp_viz.logger.setLevel(logging.INFO)

In [ ]:
lombardia_osp_viz.show_series(title='Casi ospedalizzati in Lombardia')

In [ ]:
print('Casi ospedalizzati in Lombardia:')
lombardia_osp_viz.series.tail(n=10)

In [ ]:
print('Nuovi casi giornalieri ospedalizzati in Lombardia:')
showdiff(lombardia_osp_viz.series, n=10).astype(int).to_frame()

In [ ]:
lombardia_osp_viz.show_new(title='Nuovi casi ospedalizzati in Lombardia')

# Dati delle province

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
provinces_ds = DataSet('dati-province/dpc-covid19-ita-province.csv')

def province_viz(provinces_ds, province_name):
    province_df = provinces_ds.df[provinces_ds.df['denominazione_provincia'] == province_name]
    province_tot = province_df['totale_casi'].resample('D').last()
    province_tot.name = province_name
    province_tot_viz = TimeSeriesViz(series=province_tot, last_modified=provinces_ds.last_modified)
    return province_tot_viz

def province_active_viz(provinces_ds, province_name, lookback=35):
    province_df = provinces_ds.df[provinces_ds.df['denominazione_provincia'] == province_name]
    province_tot = province_df['totale_casi'].resample('D').last()
    province_active = province_tot - province_tot.shift(lookback)
    province_active.name = province_name
    province_active_viz = TimeSeriesViz(series=province_active, last_modified=provinces_ds.last_modified)
    return province_active_viz

In [ ]:
provinces_ds.df

In [ ]:
province_df = provinces_ds.df
province_df[province_df['denominazione_regione'] == 'Lombardia']['denominazione_provincia'].unique()

In [ ]:
df_pop_raw = pd.read_csv('data/popolazione_province_2019.csv')
df_pop_raw = df_pop_raw[['ITTER107', 'Territorio', 'Value']]
df_pop = df_pop_raw[df_pop_raw['ITTER107'].str.len() == 5]

In [ ]:
def show_gr_prev(province_df, df_pop, region=False, gr_days=7, ms=10, show_trait=1, 
                 prevalence_nom = 100000, figsize=(16, 10), xlim=(None, None), ylim=(None, None)):
    region_df = province_df[province_df['denominazione_regione'] == region] if region else province_df
    provinces = region_df['denominazione_provincia'].unique()
    provinces = [p for p in provinces if not p.startswith('In fase di')]
    fig, ax = plt.subplots(figsize=figsize)

    region_pop = 0
    for province in provinces:
        data = province_df[province_df['denominazione_provincia'] == province]
        pop = df_pop[df_pop['Territorio'] == province]['Value'].sum()
        region_pop += pop
        cases = data['totale_casi']
        active_cases = cases - cases.shift(35)
        active_start = active_cases.shift(gr_days)
        growth_rate = (((active_cases - active_start) / active_start) * 100)
        prevalence = ((active_cases / pop) * prevalence_nom)
        province_plot = ax.plot(prevalence[-show_trait:], growth_rate[-show_trait:], 
                                marker='o', linestyle='-', ms=ms, label=province)
        province_color = province_plot[0].get_color()
        ax.annotate(province, (prevalence[-1], growth_rate[-1]), xytext=(ms/2, ms/2), textcoords='offset points', 
                    color=province_color, size=13)

    if region:
        region_data = region_df.groupby(region_df.index).sum()
        region_cases = region_data['totale_casi']
        active_cases = region_cases - region_cases.shift(35)
        active_start = active_cases.shift(gr_days)
        region_gr = (((active_cases - active_start) / active_start) * 100)
        region_prevalence = ((active_cases / region_pop) * prevalence_nom)
        region_plot = ax.plot(region_prevalence[-show_trait:], region_gr[-show_trait:], 
                              marker='o', linestyle='-', lw=4, ms=ms, label=region)
        region_color = region_plot[0].get_color()
        if show_trait == 1:
            ax.axvline(region_prevalence[0], c=region_color)
            ax.axhline(region_gr[0], c=region_color)
        ax.annotate(region, (region_prevalence[-1], region_gr[-1]), xytext=(5, 5), textcoords='offset points', 
                    color=region_color, size=13)
    ax.set_ylabel(f'growth rate (last {gr_days} days)')
    ax.set_xlabel(f'prevalence (active cases / {prevalence_nom} individuals)')
    title = f'Growth rate and prevalence for {region}, {cases.index[-1]:%Y-%m-%d}' \
            if region else f'Growth rate and prevalence for Italy, {cases.index[-1]:%Y-%m-%d}'
    ax.set_title(title)
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(decimals=1))
    ax.set_ylim(ylim)
    ax.set_xlim(xlim)
#     ax.legend()

In [ ]:
gr_days = 7

In [ ]:
show_gr_prev(province_df, df_pop, gr_days=gr_days, show_trait=1, figsize=(18, 14), ylim=(0, None))

In [ ]:
show_gr_prev(province_df, df_pop, 'Lombardia', gr_days=gr_days, show_trait=14)

In [ ]:
show_gr_prev(province_df, df_pop, 'Emilia-Romagna', gr_days=gr_days, show_trait=10)

In [ ]:
show_gr_prev(province_df, df_pop, 'Piemonte', gr_days=gr_days, show_trait=12)

In [ ]:
show_gr_prev(province_df, df_pop, 'Veneto', gr_days=gr_days, show_trait=14)

In [ ]:
show_gr_prev(province_df, df_pop, 'Puglia',  gr_days=gr_days, show_trait=14)

In [ ]:
show_gr_prev(province_df, df_pop, 'Lazio',  gr_days=gr_days, show_trait=10)

In [ ]:
show_gr_prev(province_df, df_pop, 'Sicilia',  gr_days=gr_days, show_trait=10)

In [ ]:
show_gr_prev(province_df, df_pop, 'Sardegna', gr_days=gr_days, show_trait=10)

In [ ]:
show_gr_prev(province_df, df_pop, 'Liguria', gr_days=gr_days, show_trait=30)

In [ ]:
province_name = 'Milano'
milano_tot_viz = province_viz(provinces_ds, province_name)
milano_tot_viz.show_series(title=f'Casi totali nella provincia di {province_name}')

In [ ]:
province_name = 'Milano'
milano_active_viz = province_active_viz(provinces_ds, province_name)
milano_active_viz.show_series(title=f'Casi attivi nella provincia di {province_name}')

In [ ]:
fig, ax = milano_tot_viz.show_new(title=f'Nuovi casi giornalieri nella provincia di Milano')
ax.set_xlim(('2020-09-01', None))

In [ ]:
milano_2d = (milano_tot_viz.series - milano_tot_viz.series.shift()).resample('2D').sum().expanding().sum()
milano_2d_viz = TimeSeriesViz(milano_2d, datetime.now())
# milano_2d_viz.show_growth_factor(title=f'Tasso di crescita dei casi a Milano (2 days)', window=4)

In [ ]:
province_name = 'Milano'
milano_tot_viz.show_growth_factor(title=f'Tasso di crescita dei casi a {province_name}', 
                                  raw=True, sma=True, ema=False, window=7)

In [ ]:
milano_tot = milano_tot_viz.series
observed = (milano_tot - milano_tot.shift())['2020-09-01':]
popt, pcov, y, argmax = show_fit(observed, func, func_name, 
                                 title=f'Nuovi casi giornalieri in Milano + {func_name} fit', 
                                 window=7, figsize=(16, 10))
K, x, t0, b = popt
print(f'Milano: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}, b = {b:.02f}')
print(f'peak as of fit: {argmax:%Y-%m-%d}')

In [ ]:
province_name = 'Bergamo'
bergamo_tot_viz = province_viz(provinces_ds, province_name)
# bergamo_tot_viz.show_series(title=f'Casi totali nella provincia di {province_name}', save_fig=True, save_csv=True)

In [ ]:
bergamo_tot_viz.show_new(title=f'Nuovi casi giornalieri nella provincia di {province_name}')

In [ ]:
(bergamo_tot_viz.series - bergamo_tot_viz.series.shift()).tail()

In [ ]:
province = 'Bergamo'
observed = (bergamo_tot_viz.series - bergamo_tot_viz.series.shift())['2020-10-01':]
popt, pcov, y, argmax = show_fit(observed, func, func_name, 
                                 title=f'Nuovi casi giornalieri in {province} + {func_name} fit',
                                 window=7)
K, x, t0, b = popt
print(f'{province}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
print(f'first day: {observed.index[0]:%Y-%m-%d}')
print(f'peak as of fit: {argmax:%Y-%m-%d}')

In [ ]:
# bergamo_tot_viz.show_growth_factor(title=f'Tasso di crescita dei casi a {province_name}', sma=False, save_fig=True, save_csv=True, ylim=(0, 5))

In [ ]:
province_name = 'Brescia'
brescia_tot_viz = province_viz(provinces_ds, province_name)
# brescia_tot_viz.show_series(title=f'Casi totali nella provincia di {province_name}', save_fig=True, save_csv=True)

In [ ]:
brescia_tot_viz.show_new(title=f'Nuovi casi giornalieri nella provincia di {province_name}')

In [ ]:
province = 'Brescia'
observed = (brescia_tot_viz.series - brescia_tot_viz.series.shift())['2020-10-01':]
popt, pcov, y, argmax = show_fit(observed, func, func_name, 
                            title=f'Nuovi casi giornalieri in {province} + {func_name} fit')
K, x, t0, b = popt
print(f'{province}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
print(f'first day: {observed.index[0]:%Y-%m-%d}')
print(f'peak as of fit: {argmax:%Y-%m-%d}')

In [ ]:
# brescia_tot_viz.show_growth_factor(title=f'Tasso di crescita dei casi a {province_name}', 
#                                    sma=False, save_fig=True, save_csv=True, ylim=(0, 5))

# Dati nazionali
## Casi totali in Italia

In [ ]:
national_totale = national.df['totale_casi']
national_totale.name = 'totali Italia'
national_all_viz = TimeSeriesViz(series=national_totale, last_modified=national.last_modified)
national_all_viz.logger.setLevel(logging.INFO)

In [ ]:
national_totale.tail()

In [ ]:
# national_all_viz.show_series(title='Casi totali in Italia', save_fig=True, save_csv=True)

In [ ]:
national_all_viz.show_new(title='Nuovi casi giornalieri in Italia', zero_min=True)
print((national_all_viz.series - national_all_viz.series.shift()).tail())

In [ ]:
total = national.df['totale_casi']
tests = national.df['casi_testati']
diff_total = (total - total.shift())
diff_tests = (tests - tests.shift())
normalized = (diff_total / (diff_tests / diff_tests[-1]))['2020-09-01':]
normalized.name = 'nuovi casi ponderati ai tamponi'
popt, pcov, fit, argmax = show_fit(normalized, func, func_name, pred=30, figsize=(16, 10),
                           title=f'Nuovi casi normalizzati giornalieri in Italia + {func_name} fit\n$n(t)=Kt^{{x}}e^{{-t/t_0}}$',
                                   window=7)
K, x, t0, b = popt
print(f'Italia: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
print(f'first day: {normalized.index[0]:%Y-%m-%d}')
print(f'peak as of fit: {argmax:%Y-%m-%d}')

In [ ]:
observed = (national_totale - national_totale.shift())['2020-09-01':]
popt, pcov, y, argmax = show_fit(observed, func, func_name, 
                                 title=f'Nuovi casi giornalieri in Italia + {func_name} fit', 
                                 figsize=(16, 10), pred=15, window=7)
K, x, t0, b = popt
print(f'Italia: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
print(f'first day: {observed.index[0]:%Y-%m-%d}')
print(f'peak as of fit: {argmax:%Y-%m-%d}')

In [ ]:
observed

In [ ]:
# total = national.df['totale_casi']
# tests = national.df['tamponi']
# diff_total = (total - total.shift())
# diff_tests = (tests - tests.shift())
# normalized_total = (diff_total / (diff_tests / diff_tests.max()))[1:]
# normalized_total['2020-03-09'] = (normalized_total['2020-03-08'] + normalized_total['2020-03-10']) / 2
# normalized_new.name = 'nuovi casi ponderati ai tamponi'
# popt, pcov, fit = show_fit(normalized_new, func, func_name,
#                          title=f'Nuovi casi normalizzati giornalieri in Italia + {func_name} fit\n$n(t)=Kt^{{x}}e^{{-t/t_0}}$')
# K, x, t0 = popt
# print(f'{region}: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
# print(f'first day: {normalized_new.index[0]:%Y-%m-%d}')
# print(f'peak as of fit: {normalized_new.index[np.argmax(fit)]:%Y-%m-%d}')

In [ ]:
national_all_viz.show_growth_factor(title='Tasso di crescita per i casi totali in Italia', 
                                    raw=False, sma=True, ema=False, window=7)

## Casi decessi in Italia

In [ ]:
deaths = national.df['deceduti']
deaths.name = 'deceduti Italia'
deaths_viz = TimeSeriesViz(series=deaths, last_modified=national.last_modified)
deaths_viz.logger.setLevel(logging.INFO)

In [ ]:
# deaths_viz.show_series(title='Casi deceduti in Italia', save_fig=True, save_csv=True)

In [ ]:
# deaths_viz.show_new(title='Nuovi casi deceduti in Italia', save_fig=True, save_csv=True)

In [ ]:
observed = (deaths - deaths.shift())['2020-10-01':]
popt, pcov, y, argmax = show_fit(observed, func, func_name, pred=14,
                                 title=f'Nuovi deceduti giornalieri in Italia + {func_name} fit', 
                                 figsize=(16, 10))
K, x, t0, b = popt
print(f'Italia: K = {K:.08f}, x = {x:.02f}, t0 = {t0:.02f}')
print(f'first day: {observed.index[0]:%Y-%m-%d}')
print(f'peak as of fit: {argmax:%Y-%m-%d}')

In [ ]:
observed.tail()

In [ ]:
# deaths_viz.show_growth_factor(title='Tasso di crescita dei deceduti in Italia', sma=False, save_fig=True, save_csv=True)

## Casi in terapia intensiva in Italia

In [ ]:
ti = national.df['terapia_intensiva']
ti.name = 'ti Italia'
ti_viz = TimeSeriesViz(series=ti, last_modified=national.last_modified)
ti_viz.logger.setLevel(logging.INFO)

In [ ]:
ti_viz.show_series(title='Casi in terapia intensiva in Italia')

In [ ]:
ti_viz.show_new(title='Nuovi casi in terapia intensiva in Italia')

In [ ]:
all_serious = national.df['ricoverati_con_sintomi'] +  national.df['terapia_intensiva']
all_serious.name = 'tutti casi seri Italia'
all_serious_viz = TimeSeriesViz(series=all_serious, last_modified=national.last_modified)
all_serious_viz.logger.setLevel(logging.INFO)

In [ ]:
all_serious_viz.show_series(title='Tutti casi seri (ricoverati + ti) in Italia')

In [ ]:
all_serious_viz.show_new(title='Nuovi casi seri (ricoverati + ti) in Italia')

In [ ]:
# regions_df_totale = regional_ds.df.pivot(columns='denominazione_regione', values='totale_casi').resample('D').last()
# regions_df_totale_diff = (regions_df_totale - regions_df_totale.shift())
# selected_regions = ['Lombardia', 'Emilia Romagna', 'Veneto']
# selected_regions = ['Lombardia']
# show_gr(regions_df_totale_diff[selected_regions], raw=True, sma=False, ema=True,
#         title=f'Tasso di cressita dei casi totali in alcuni Regioni\n'a
#               f'(dati aggiornati: {regional_ds.last_modified:%d/%m/%Y %H:%M:%S})')